In [0]:
import pandas as pd
import numpy as np


In [0]:
cbp = np.load('cat_catboost_probas.npy')
gp = np.load('grouped_probas.npy')
mep = np.load('mn_enc_probas.npy')
nop = np.load('node_probas.npy')

In [0]:
edges = pd.read_csv('edges.csv')
vertices = pd.read_csv('new_vertices.csv')


In [0]:
ids = pd.read_csv('ids.csv')

In [0]:
all_connections = pd.read_csv('all_connections')

In [0]:
index = all_connections.id_1
connections = all_connections.connection
inc = dict(zip(index, map(eval, connections)))

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
import xgboost as xgb

In [0]:
import tqdm

In [0]:
res = np.zeros((100, 1534749))
for i in tqdm.tqdm(range(100)):
  indexes = edges[(edges['id_1']==ids.id.iloc[i])|(edges['id_2']==ids.id.iloc[i])].index
  right = pd.Series(np.zeros(len(vertices)))
  right[pd.Series(inc[ids.id.iloc[i]])-1] = 1
  table = pd.DataFrame({'cbp': cbp[i], 'gp':gp[i], 'mep': mep[i], 'target': right})
  train = table.drop(columns='target')
  target = table['target']
  w = table[table.target==0].target.count()/table[table.target==1].target.count()
  param_dist = {'max_depth':6, 'n_estimators':500, 'n_jobs':-1, 'random_state': 42, 'lambda': 0.2, 'scale_pos_weight':w}
  clf = xgb.XGBClassifier(**param_dist,tree_method='gpu_hist', gpu_id=0)

  
  # clf = LogisticRegression(random_state=42, solver = 'liblinear')
  clf.fit(train, target)
  q = clf.predict_proba(train)
  res[i] = q[:, 1]

100%|██████████| 100/100 [13:18<00:00,  7.99s/it]


In [0]:
np.save('xgb65.npy', res, allow_pickle=True)